In [45]:
#Library
import requests
import json
from datetime import datetime
import math
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 1000)
pd.set_option("display.precision", 12)
import warnings
warnings.filterwarnings('once')

In [46]:
def get_data(endpointApi, date=None, full_response=False, columns=list()):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    payload = {'date': date, 'fullResponse': full_response}
    res = requests.get(URL + endpointApi,# + columns_to_str(columns),
                      params=payload,
                      auth=AUTH,
                      verify=False)
    return res.content.decode('utf-8')

In [47]:
def put_data(endpointApi, content):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    res = requests.post(url=URL + endpointApi,# + columns_to_str(columns),
                      data=content,
                      auth=AUTH,
                      verify=False)
    return res.content.decode('utf-8')

Conversion chaque valeur en le type approprié :

In [48]:
def convert_currency(value, df_convert_currency):
    value, currency_name = value.split(' ')
    rate = currencyRateToEUR(currency_name, df_currency_rate)
    
    new_value = float(value.replace(',', '.')) * rate
    return new_value

def convert_datetime(value):
    return datetime.strptime(value, '%Y-%m-%d').date()

def convert_float(value):
    return float(value.replace(',', '.'))

def convert_int(value):
    return int(value)

def convert_boolean(value):
    return json.loads(value)

def df_convert_type(df, df_currency_rate):
        for col in df.columns:
            convert_values = []
            for elt in df[col]:
                if elt is np.nan:
                    convert_values.append(np.nan)
                    continue

                elt_type = elt['type']
                elt_value = elt['value']
                
                if elt_type == 'currency_value':
                    elt_value = convert_currency(elt_value, df_currency_rate)
                elif elt_type == 'date':
                    elt_value = convert_datetime(elt_value)
                elif elt_type in ['double', 'percent']:
                    elt_value = convert_float(elt_value)
                elif elt_type in ['asset', 'int32']:
                    elt_value == convert_int(elt_value)
                elif elt_type == 'boolean':
                    elt_value = convert_boolean(elt_value)
                elif elt_type not in ['asset_type', 'string', 'asset_currency', 'date-time',
                                     'asset_sub_type', 'asset_status', 'asset_quote_type',
                                     'liquidity_algorithm', 'portfolio_lock_mode', 'portfolio_type']:
                    print(elt)
                convert_values.append(elt_value)
            df[col] = convert_values
        return df

Création de la table des taux en EUR : 
 
[Est-ce qu'on donne la posibilité de changer la currency dst en fonction de la currency du portfolio ?]

In [49]:
def getCurrencyRate():
    d = []
    currency_json = json.loads(get_data('currency'))
    for currency in currency_json:
        currency_id = currency.get('id')
        rate = get_data('currency/rate/{}/to/EUR'.format(currency_id))
        if len(rate) == 0:
            continue
        rate_value = json.loads(rate)['rate']['value']
        d.append([currency_id, convert_float(rate_value)])
    return pd.DataFrame(d, columns=['currency src', 'rate'])
    
df_currency_rate = getCurrencyRate()
df_currency_rate

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,currency src,rate
0,USD,0.851208716377
1,EUR,1.000000000000
2,JPY,0.008066467694
3,BEF,0.024789352477
4,CYP,1.708601441376
...,...,...
23,LUF,0.024789352477
24,VAL,0.000516456899
25,SML,0.000516456899
26,EUr,0.010000000000


[TEST] Taux de convertion entre USD et EUR :

In [50]:
def currencyRateToEUR(cur_src, df_currency_rate):
    return df_currency_rate[df_currency_rate['currency src'] == cur_src]['rate'].values[0]
rate = currencyRateToEUR('USD', df_currency_rate)
rate


0.851208716377

[TEST] La database de tous les asset :

In [51]:
data = get_data('asset/')
df = pd.read_json(data)
df_ETF = df_convert_type(df, df_currency_rate)
df_ETF

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,MARKET_PLACE_MIC,LAST_GROSS_VALUE,LAST_QUOTE_DATE,MARKET_PLACE_CURRENCY,BLOOMBERG_CODE,CODE_CIC,CATEGORY_CIC,CREATION_DATE,GICS_SUB_INDUSTRY,FIRST_QUOTE_DATE,LAST_QUOTE_SOURCE,VL_DELAY,SUB_CATEGORY_CIC,MARKET_CAP_CATEGORY,SUB_SECTOR,INDEXED_RATE,GRAPH_PAST_WEEK,GRAPH_LAST_YEAR,PORT_LINE_COUNT,MARKET_PLACE_CITY,IS_PORTFOLIO,DAILY_PRICE_COUNT,EMIR_CLEARING_ID_TYPE,BLOOMBERG_ID,MARKET_PLACE,LAST_QUOTES_UPDATE_DATE,IS_ACCESSIBILITY_PUBLIC,MARKET_PLACE_DESC,GRAPH_PERIOD,INDUSTRY,GRAPH_PAST_YEAR,REST_UNIQUE_NUMBER,asset_custom_classification_bloomberg,COUNTRY_CIC,GLOBAL_TYPE,ASSET_OR_MARKET_PLACE_COUNTRY,LAST_CLOSE_VALUE_IN_CURR,MARKET_PLACE_TELEKURS,COUNTRY,LABEL,LAST_CLOSE_VALUE,MODIFICATION_DATE,ASSET_DATABASE_ID,GICS_INDUSTRY_GROUP,PORT_CONTENT_COUNT,IS_FUND_FED,COUNTRY_ISO_3,IS_STRUCTURED_PRODUCT,FINAL_UNDERLYING,UNDERLYING_OR_ASSET_IF_UNDERLYING_NOT_AVAILABLE,GICS_SECTOR,GRAPH_LAST_6_MONTH,asset_attr_init,quote_value_type,ISIN_CODE,MARKET_PLACE_OR_ASSET_COUNTRY,MARKET_PLACE_COUNTRY,IS_DEPOSITARY_QUOTES,ASSET_STATUS,REST_OBJECT_ID,IS_EURONEXT,SEDOL_CODE,IS_COMPOSITE,FINAL_UNDERLYING_TYPE,FLOATING_VALUE,IS_PUBLICLY_TRADED,PAY_ACCOUNT_ON_DEFAULT,IS_FUND_SAVE_SENS,GICS_INDUSTRY,SECTOR,CURRENCY,SUB_TYPE,TYPE,LIQUIDITY_ALGO,QUOTE_TYPE,IS_EUROLAND,STOCK_TYPE,MARKET_PLACE_TYPE,CITY_OR_ACRONYM_OR_NAME,SUP_SECTOR,asset_custom_classification_Tab. de bord GSM,ASSET PRICE MODE,ELIGIBLE_PEA_PME,ELIGIBLE_PEA,ASSET_MARKET_SUB_GROUP,asset_fund_info_asset_class_focus,asset_fund_info_strategy,asset_fund_info_maximum_front_end_fee,BOND_YTM_COMPUTATION_MODE,asset_fund_info_bloomberg_ticker,BOND_CREDIT_RISK_COMPUTATION_MODE,asset_fund_info_is_feeded,asset_fund_info_authorized_for_retail_investors,asset_fund_info_fund_leverage,asset_fund_info_objective_long,asset_fund_info_is_euro_ucits,BOND_RISK_COMPUTATION_MODE,asset_fund_info_administrator,TRANSPARENCY_LIST,asset_fund_info_isin_code,asset_fund_info_minimum_investment_shares,asset_fund_info_depositary,asset_fund_info_manager_stated_fee,BOND_WAL_COMPUTATION_MODE,asset_fund_info_cut_off,asset_fund_info_maximum_back_end_fee,asset_fund_info_company,asset_fund_info_vl_date,IS_REAL_ESTATE_FUND,asset_fund_info_manager_stated_perf_fee,asset_fund_info_auditor,FUND_TYPE,asset_fund_info_fund_type,asset_fund_info_valorisation_frequency,asset_fund_info_creation_date,BOND_WAM_COMPUTATION_MODE,asset_fund_info_pct_max_opcvm,asset_fund_info_initial_minimum_subscription,MIN_BUY_AMOUNT,asset_fund_info_fund_open_pymt_shr,asset_fund_info_amf_classification,asset_fund_info_decimalisation,asset_fund_info_amf_date,asset_fund_info_minimum_subscription,asset_fund_info_day_cut_off,asset_fund_info_vl_known,asset_fund_info_geographical_focus,asset_fund_info_subscription_committee,MIN_BUY_INITIAL_AMOUNT,asset_fund_info_abide_with_the_four_criterias,REUTERS_CODE,asset_fund_info_reuters_code,asset_fund_info_subscription_frequency,asset_fund_info_redemption_frequency,VALOREN_CODE,asset_fund_info_valoren_code,asset_fund_info_manager,asset_fund_info_redemption_committee,asset_fund_info_advised_horizon,FUND_REF_IDX,asset_fund_info_comanager,HAS_TOBIN_TAX,CUSIP_CODE,LOCAL_CODE,asset_fund_info_rate_entry,asset_fund_info_management_style,asset_fund_info_rate_output,asset_fund_info_rate_entry_sdg,asset_fund_info_rate_output_sdg,MARKET_QUOTE_IDENTIFIER,asset_fund_info_manager_committee,PORT_ASSET_CONTENT_COUNT,INDEX_TYPE,QUOTE_PLACE,TELEKURS_QUOTE_PLACE_CODE,IS_SRD,PORT_BASE_NAV,PORT_VL,REAL_TIME_PORT_CASH,PORT_CLEAN_MODE,PORT_CASH,PORT_NET_EX_CASH_EQUIVALENT,PORT_FEED_CODE,NB_COMPTE_ON_PORTFOLIO,PORT_CURRENCY_HEDGE,EAI_FRONTPORT_CASH_BY_CRCY,LOCK_MODE,PORT_CASH_AVAILABLE,EAI_BACKPORT_CASH_BY_CRCY,EAI_PORTFOLIO_FEED_NAME,PORT_FEED_TYPE,PORT_NET_EX_CASH,PORT_IS_HISTORICAL,PORT_NEED_RECONCILIATION,PORT_TYPE,PORT_CASH_EQUIVALENTS,PORT_STATUS,EAI_PORT_AVAILABLE_CASH,PRICER_MODE,VL_DELAY_VALUE,PORT_RATING_MOODYS_SHORT_TERM,PORT_PERCENT_EQUIVALENT,PORT_RATING_SP_SHORT_TERM,PORT_FIRST_TRANSACTION,PORT_RATING_MOODYS,PORT_RATING_SP,PORT_PERCENT_

Récupération de la liste d'actifs en base :

[On prend juste ces valeurs comme dans l'exemple ou toutes les valeurs des assets ?]

In [52]:
def get_df_asset(asset_date, df_currency_rate, asset_id='ASSET_DATABASE_ID', asset_label='LABEL',
                 asset_type='TYPE', asset_last_close_value='LAST_CLOSE_VALUE_IN_CURR'):
    data = get_data('asset?columns={}&columns={}&columns={}&columns={}&date={}'
                   .format(asset_id, asset_label, asset_type, asset_last_close_value, asset_date))
    df_asset = pd.read_json(data)
    df_asset = df_convert_type(df_asset, df_currency_rate)
    return df_asset

df_ETF = get_df_asset('2013-06-14', df_currency_rate)
df_ETF

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,LAST_CLOSE_VALUE_IN_CURR,TYPE,LABEL,ASSET_DATABASE_ID
0,12.120000000000,STOCK,1&1 DRILLISCH AG,1845
1,44.910000000000,STOCK,1000MERCIS,1846
2,30.457000000000,FUND,1818-VEGA EUR CO,2122
3,1342.230000000000,FUND,29 HAUSSMANN CROISSANCE EUROPE D,2123
4,NaN,FUND,29 HAUSSMANN EQUILIBRE,2124
...,...,...,...,...
475,8.530000000000,STOCK,YAMANA GOLD INC,1879
476,7.200000000000,STOCK,YMAGIS SA,1997
477,2.490000000000,STOCK,ZCCM INVESTMEN - B,1991
478,26.217228464412,STOCK,ZOETIS INC,2106


Différence entre LAST_CLOSE_VALUE_IN_CURR et LAST_CLOSE_VALUE ??

Récupération des valeurs de cotation:

Cotation de l'actif 1845 en fonction de l'id et de la date de debut et fin :

In [53]:
def get_cotation_values(asset_id, start_date, end_date, df_currency_rate):
    data = get_data('asset/{}/quote?start_date={}&end_date={}'
                    .format(asset_id, start_date, end_date))
    df_cotation = pd.read_json(data)
    df_cotation = df_convert_type(df_cotation, df_currency_rate)
    return df_cotation

df_cotation = get_cotation_values('1845', '2013-06-14', '2019-05-31', df_currency_rate)
df_cotation
    

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,date,nav,gross,volume,high,real_close_price,feed_source,low,ask,asset,pl,bid,close,return,open
0,2013-06-14,12.120000000000,12.120000000000,312048.0,12.170,12.120000000000,-10.0,12.010,12.130,1845,0.110000000000,12.075,12.120000000000,0.009159034138,12.010
1,2013-06-17,12.610000000000,12.610000000000,447491.0,12.650,12.610000000000,-10.0,12.085,12.655,1845,0.490000000000,12.575,12.610000000000,0.040429042904,12.085
2,2013-06-18,12.880000000000,12.880000000000,300052.0,12.880,12.880000000000,-10.0,12.555,12.905,1845,0.270000000000,12.825,12.880000000000,0.021411578113,12.605
3,2013-06-19,12.815000000000,12.815000000000,260988.0,12.960,12.815000000000,-10.0,12.710,12.805,1845,-0.065000000000,12.795,12.815000000000,-0.005046583851,12.865
4,2013-06-20,12.370000000000,12.370000000000,263978.0,12.775,12.370000000000,-10.0,12.370,12.435,1845,-0.445000000000,12.355,12.370000000000,-0.034724931721,12.710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519,2019-05-27,36.006862268298,36.006862268298,NaN,NaN,36.006862268298,-10.0,NaN,NaN,1845,0.539152338935,NaN,36.006862268298,0.015201216543,NaN
1520,2019-05-28,36.067253192136,36.067253192136,NaN,NaN,36.067253192136,-10.0,NaN,NaN,1845,0.060390923838,NaN,36.067253192136,0.001677205956,NaN
1521,2019-05-29,36.311697245690,36.311697245690,NaN,NaN,36.311697245690,-10.0,NaN,NaN,1845,0.244444053554,NaN,36.311697245690,0.006777451342,NaN
1522,2019-05-30,35.796349125318,35.796349125318,NaN,NaN,35.796349125318,-10.0,NaN,NaN,1845,-0.515348120373,NaN,35.796349125318,-0.014192344601,NaN


Récupération de la liste des ratios :

In [54]:
def get_ratio(df_currency_rate):
    data = get_data('ratio')
    df_ratio = pd.read_json(data)
    #df_ratio = df_convert_type(df_ratio, df_currency_rate)
    return df_ratio

df_ratio = get_ratio(df_currency_rate)
df_ratio

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,id,type,name,is_benchmark_needed,is_percent
0,11,Ratio,Correlation,True,False
1,7,Ratio,Exposition action,False,True
2,15,Ratio,Exposition action,False,True
3,8,Ratio,RDT,False,True
4,13,Ratio,Rendement,False,True
5,9,Ratio,Rendement_annualise,False,True
6,12,Ratio,Sharpe,False,False
7,14,Ratio,VaR historique,False,True
8,10,Ratio,Volatilite,False,True


[FIX portfolio ID] Récupération de la composition d’un portefeuille :

In [55]:
def get_portfolio(portfolio_id):
    data = get_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id))
    df_portfolio = df_convert_type(data)
    return df_portfolio
    #keep seulement list assets ?

portfolio = get_portfolio(564)
portfolio

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


TypeError: df_convert_type() missing 1 required positional argument: 'df_currency_rate'

[FIX portfolio ID] MAJ de la composition d’un portefeuille :


In [56]:
def portfolio_json_assets(assets):
    json = ''
    for asset in assets:
        json += '{{"asset":{{"asset": {}, "quantity": {}}}}},'.format(asset["asset"], asset["quantity"])
    return json[:-1]
    
def seralize_portfolio_content(df_portfoliot, assets):
    label = "EPITA_PTF_1"
    currency = "EUR"
    date = "2013-06-14"
    assets = portfolio_json_assets(assets) 
    res = '[{{"label": "{}", "currency": {{"code": "{}"}}, "type": "front", "values": {{"{}": [{}]}}}}]'.format(label, currency, date, assets)
    return res

def deseralize_portfolio_content(json_portfolio):
    json_portfolio = pd.json_normalize(json_portfolio)
    
    cols = [col for col in json_portfolio.columns if col.split('.')[0] == 'values']
    json_portfolio = json_portfolio[cols]
    json_portfolio.columns = json_portfolio.columns.map(lambda x: x.split(".")[0])

    json_assets = json_portfolio['values'][0]
    assets = []
    for asset in json_assets:
        assets.append(asset['asset'])
    return assets

json_portfolio = json.loads(seralize_portfolio_content('', [{'asset': 42, 'quantity': 2}, {'asset': 1, 'quantity': 1}]))
print(json_portfolio)

assets = deseralize_portfolio_content(json_portfolio)
assets

[{'label': 'EPITA_PTF_1', 'currency': {'code': 'EUR'}, 'type': 'front', 'values': {'2013-06-14': [{'asset': {'asset': 42, 'quantity': 2}}, {'asset': {'asset': 1, 'quantity': 1}}]}}]


[{'asset': 42, 'quantity': 2}, {'asset': 1, 'quantity': 1}]

In [57]:
def put_portfolio(portfolio_id, assets):
    put_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id), assets)


assets = [{'asset': 42, 'quantity': 2}, {'asset': 1, 'quantity': 1}]
content = seralize_portfolio_content('', assets)

put_portfolio(564, content)

[TODO] Calcul de ratios :

In [58]:
def volatility(returns, nb_period_per_year): #X : col du df
    return returns.std() * np.sqrt(nb_period_per_year)

volatility(df_cotation['close'].values, 365)

249.6741996565642

In [59]:
def beta(X, Y):
    return np.cov(X, Y) / np.var(Y)

In [60]:
def correlation(Ra, Rb):
    return np.cov(Ra, Rb) / np.sqrt(np.var(Ra) * np.var(Rb))

In [61]:
def rendement_Nav(Nav, Nav_prev):
    return (Nav - Nav_prev) / Nav_prev
    #R - R.shift(1))/R.shift(1)

In [62]:
def rendement_annual():
    

SyntaxError: unexpected EOF while parsing (<ipython-input-62-25ba0d1eb457>, line 2)

In [63]:
def return_rate(X_end, X_begin, nb_days):
    return (1 + (X_end - X_begin) / X_begin)**(365/nb_days) - 1

In [64]:
def sharpe_ratio(return_rate, Rf, nb_period_per_year):
    volatility = volatility(returns, nb_period_per_year)
    return (return_rate - Rf) / volatility

In [65]:
def value_at_risk(Ra, Threshold):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-65-3b3248e14795>, line 2)

In [ ]:
def action_exposure():